In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
kaggle_json_path = '/content/drive/MyDrive/Colab Notebooks/Kaggle/kaggle.json'

In [3]:
!pip install kaggle

In [4]:
import shutil

!mkdir /root/.kaggle

shutil.copy(kaggle_json_path, '/root/.kaggle')

!chmod 600 ~/.kaggle/kaggle.json

'/root/.kaggle/kaggle.json'

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nicolaswattiez/skillbuilder-data-2009-2010")

print("Path to dataset files:", path)

100%|██████████| 629M/629M [00:29<00:00, 22.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nicolaswattiez/skillbuilder-data-2009-2010/versions/4


Import Packages

In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

In [8]:
data = pd.read_csv('/root/.cache/kagglehub/datasets/nicolaswattiez/skillbuilder-data-2009-2010/versions/4/2012-2013-data-with-predictions-4-final.csv')

In [9]:
# Randomly sample 100,000 rows
train_df = data.sample(n=100000, random_state=42)

In [10]:
column_names = train_df.columns

print(column_names)

Index(['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id',
       'assistment_id', 'start_time', 'end_time', 'problem_type', 'original',
       'correct', 'bottom_hint', 'hint_count', 'actions', 'attempt_count',
       'ms_first_response', 'tutor_mode', 'sequence_id', 'student_class_id',
       'position', 'type', 'base_sequence_id', 'skill_id', 'teacher_id',
       'school_id', 'overlap_time', 'template_id', 'answer_id', 'answer_text',
       'first_action', 'problemlogid', 'Average_confidence(FRUSTRATED)',
       'Average_confidence(CONFUSED)', 'Average_confidence(CONCENTRATING)',
       'Average_confidence(BORED)'],
      dtype='object')


Preprocessing The Dataset

Dropping Columns

In [11]:
train_df.drop(columns=['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id', 'assistment_id', 'start_time', 'end_time', 'problem_type', 'original', 'bottom_hint', 'actions', 'ms_first_response',
       'tutor_mode', 'sequence_id', 'student_class_id', 'position', 'type', 'base_sequence_id', 'skill_id', 'teacher_id',
       'school_id', 'answer_id', 'answer_text', 'template_id', 'first_action', 'problemlogid', 'Average_confidence(FRUSTRATED)', 'Average_confidence(CONFUSED)', 'Average_confidence(CONCENTRATING)',
       'Average_confidence(BORED)'], inplace=True)

In [12]:
column_names = train_df.columns
print(column_names)

Index(['correct', 'hint_count', 'attempt_count', 'overlap_time'], dtype='object')


In [29]:
train_df['attempt_and_hint'] = train_df['attempt_count'] * train_df['hint_count']

train_df['difficulty_score'] = (
    (1 - train_df['correct']) * 2 +  # Higher weight for incorrect attempts
    (0.8 * train_df['attempt_count']) +  # Higher weight for more attempts
    (0.5 * train_df['hint_count']) +  # Increased weight for hint usage
    (0.3 * train_df['overlap_time'])  # Overlap time as a factor
)

train_df['difficulty_score'] = 1 + 9 * (train_df['difficulty_score'] - min_score) / (max_score - min_score)

Preprocessing

In [24]:
from sklearn.model_selection import train_test_split

X = train_df[['correct', 'attempt_count', 'hint_count', 'overlap_time']]
y = train_df['difficulty_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])


Training set size: 80000
Test set size: 20000


In [47]:
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')  # Linear activation for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16)

loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error: {mae}")


Epoch 1/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 1384936.7500 - mae: 343.6760 - val_loss: 0.9580 - val_mae: 0.9773
Epoch 2/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 1886.1847 - mae: 7.4677 - val_loss: 0.4858 - val_mae: 0.6949
Epoch 3/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 191.8353 - mae: 1.3659 - val_loss: 0.0335 - val_mae: 0.1751
Epoch 4/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 30.2184 - mae: 0.2550 - val_loss: 0.0031 - val_mae: 0.0418
Epoch 5/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 3.2652 - mae: 0.0798 - val_loss: 0.0028 - val_mae: 0.0344
Epoch 6/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 159.6812 - mae: 0.1248 - val_loss: 0.0028 - val_mae: 0.0320
Epoch 7/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 27.6808 - mae: 0.0852 - val_loss: 0.0028 - val_mae: 0.0335
Epoch 8/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 5.6402 - mae: 0.0634 - val_loss: 0.0028 - val_mae: 0.0331
Epoch 9/50
4000

In [51]:
import numpy as np

new_data = np.array([[0, 6, 3, 20000]])

predicted_difficulty = model.predict(new_data)

print(f"Predicted Difficulty: {predicted_difficulty}")

desired_min = 0
desired_max = 10

output_min = np.min(predicted_difficulty)
output_max = np.max(predicted_difficulty)

if output_min == output_max:
    print("Warning: All predicted values are the same. Scaling will not work properly.")
    scaled_difficulty_score = np.array([[5]])
else:
    scaled_difficulty_score = (predicted_difficulty - output_min) / (output_max - output_min) * (desired_max - desired_min) + desired_min

    scaled_difficulty_score = np.clip(scaled_difficulty_score, 0, 10)

print(f"Scaled Difficulty Score: {scaled_difficulty_score[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Difficulty: [[1.0396135]]
Scaled Difficulty Score: 5.00


In [54]:
model.save('model.h5')